In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
import numpy as np
from keras.utils import to_categorical

In [12]:
df = pd.read_csv("data-cleaned.csv")
df.drop('Unnamed: 0',inplace=True,axis=1)
df

,L_label,a_label,b_label,Construction,BackingLayer2nd,PatternLayer3rd,L_weft,a_weft,b_weft,WeftDenier,WeftYarnType,WarpColor,WeavingMachine
0,38.00407,36.253840,26.323170,DD164,White,B,37.525,37.708,26.546,75D,1211,Black,Taffeta
1,37.67931,36.386490,26.184560,DD164,White,R,37.525,37.708,26.546,75D,1211,Black,Taffeta
2,37.88121,36.205170,26.034750,DD164,White,Y,37.525,37.708,26.546,75D,1211,Black,Taffeta
3,38.14394,37.289590,26.690500,DD164,White,B,37.525,37.708,26.546,75D,1211,White,Taffeta
4,38.17764,37.941370,26.850100,DD164,White,R,37.525,37.708,26.546,75D,1211,White,Taffeta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59476,84.92196,-5.782575,1.478362,DD164,White,R,84.432,-8.272,2.727,75D,1211,White,Taffeta
59477,86.67738,-6.509423,4.290485,DD164,White,Y,84.432,-8.272,2.727,75D,1211,White,Taffeta
59478,81.18523,-5.026460,0.591886,DD164,White,B,84.432,-8.272,2.727,75D,1211,Black,Taffeta
59479,81.17082,-5.086213,0.662887,DD164,White,R,84.432,-8.272,2.727,75D,1211,Black,Taffeta


We are going to train only for the target L_weft (as there are 3 targets a,b and L)

In [13]:
df.drop('a_weft',inplace=True,axis=1)
df.drop('b_weft',inplace=True,axis=1)

y = df.L_weft

x = df.drop('L_weft',axis=1)

x.drop(columns=["Construction","BackingLayer2nd","PatternLayer3rd","WeftDenier","WarpColor","WeavingMachine"],axis=1,inplace=True)

x

,L_label,a_label,b_label,WeftYarnType
0,38.00407,36.253840,26.323170,1211
1,37.67931,36.386490,26.184560,1211
2,37.88121,36.205170,26.034750,1211
3,38.14394,37.289590,26.690500,1211
4,38.17764,37.941370,26.850100,1211
...,...,...,...,...
59476,84.92196,-5.782575,1.478362,1211
59477,86.67738,-6.509423,4.290485,1211
59478,81.18523,-5.026460,0.591886,1211
59479,81.17082,-5.086213,0.662887,1211


In [14]:


#Transforming our data into np.float32 because it's easier and better for our model
x = np.asarray(x).astype(np.float32)

x

array([[ 3.800407e+01,  3.625384e+01,  2.632317e+01,  1.211000e+03],
       [ 3.767931e+01,  3.638649e+01,  2.618456e+01,  1.211000e+03],
       [ 3.788121e+01,  3.620517e+01,  2.603475e+01,  1.211000e+03],
       ...,
       [ 8.118523e+01, -5.026460e+00,  5.918860e-01,  1.211000e+03],
       [ 8.117082e+01, -5.086213e+00,  6.628871e-01,  1.211000e+03],
       [ 8.092577e+01, -5.045742e+00,  6.496191e-01,  1.211000e+03]],
      dtype=float32)

In [15]:
#Fixing the random state to 42 (arbitrary value)
X_train, X_test, y_train, y_test=train_test_split(x, y, test_size=0.1, random_state=42)



y_train.shape, X_train.shape

((53532,), (53532, 4))

In [16]:
print("--Make model--")
#Adding Dense layers with relu activation to increase the model's accuracy
model = tf.keras.models.Sequential([
  Dense(10, input_shape=(4,)),
  Dense(256, activation='relu'),
  Dense(256, activation='sigmoid'),
  Dense(128, activation='sigmoid'),
  Dense(64, activation='sigmoid'),
  Dense(1, activation='sigmoid')
])

#Changing the model optimizer to adam and using mean squared error and mean absolute percentage error because it's a regression problem
model.compile(optimizer='adam', loss='mse',metrics=['mean_absolute_percentage_error'])

print("--Fit model--")
model.fit(X_train, y_train, epochs=1, verbose=2)

--Make model--
--Fit model--
1673/1673 - 2s - loss: 3817.5544 - mean_absolute_percentage_error: 98.1935 - 2s/epoch - 1ms/step


In [17]:
print("--Evaluate model--")
model_loss1,model_abs_error1 = model.evaluate(X_train,  y_train, verbose=2)
model_loss2,model_abs_error2 = model.evaluate(X_test,  y_test, verbose=2)
print("Train loss:",model_loss1,"\nTest loss :",model_loss2)
print("Train MAPE:",model_abs_error1,"%","\nTest MAPE :",model_abs_error2,"%")

--Evaluate model--
1673/1673 - 1s - loss: 3817.3787 - mean_absolute_percentage_error: 98.1906 - 1s/epoch - 669us/step
186/186 - 0s - loss: 3800.5806 - mean_absolute_percentage_error: 98.1844 - 131ms/epoch - 702us/step
Train loss: 3817.378662109375 
Test loss : 3800.58056640625
Train MAPE: 98.19061279296875 % 
Test MAPE : 98.18440246582031 %
